<p align="center">
  <img src="https://cdn.icon-icons.com/icons2/2592/PNG/512/spotify_logo_icon_154469.png" width="200">
</p>


# Introdução

O Spotify é um serviço líder de streaming de música que revolucionou a forma como interagimos com a música. Com milhões de faixas disponíveis e uma base de usuários global, o Spotify não apenas oferece uma plataforma para artistas divulgarem suas obras, mas também gera uma quantidade imensa de dados sobre preferências musicais, tendências e hábitos de escuta. Analisar esses dados pode revelar insights valiosos sobre a indústria musical, as preferências dos ouvintes e a dinâmica do sucesso musical.

Este projeto visa explorar o dataset do top hits do Spotify, utilizando poderosas bibliotecas de análise de dados, como a Polars para manipulação de dados e a Plotly para criação de visualizações interativas. Através deste estudo, buscamos entender melhor as características das músicas que alcançam o topo das paradas, identificar padrões e tendências ao longo do tempo e oferecer uma experiência visual rica que torne essas descobertas acessíveis a todos.

In [5]:
# Importação de bibliotecas para análise

import polars as pl
import plotly.express as px

# importação de arquivo com funções de análise

import utils as ut

## Dicionário de dados:

Dicionário de dados fornecido pelo [Kraggle](https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019)

| Coluna           | Tipo de Dado | Descrição                                                                                                                                                              |
|------------------|--------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| artist           | String       | Nome do Artista.                                                                                                                                                       |
| song             | String       | Nome da Faixa.                                                                                                                                                         |
| duration_ms      | Int64        | Duração da faixa em milissegundos.                                                                                                                                     |
| explicit         | Boolean      | Indica se a letra ou conteúdo de uma música ou vídeo musical contém um ou mais critérios que podem ser considerados ofensivos ou inadequados para crianças.           |
| year             | Int64        | Ano de lançamento da faixa.                                                                                                                                            |
| popularity       | Int64        | Quanto maior o valor, mais popular é a música. Os valores variam de 0 a 100.                                                                                           |
| danceability     | Float64      | Descreve quão adequada uma faixa é para dançar com base em uma combinação de elementos musicais. Um valor de 0.0 é o menos dançável e 1.0 é o mais dançável.          |
| energy           | Float64      | Uma medida de 0.0 a 1.0 que representa uma medida perceptual de intensidade e atividade.                                                                               |
| key              | Int64        | A tonalidade em que a faixa está. Inteiros mapeiam para notas usando a notação padrão de Classe de Altura. Por exemplo, 0 = C, 1 = C♯/D♭, etc. -1 indica sem tonalidade detectada. |
| loudness         | Float64      | A loudness geral de uma faixa em decibéis (dB), média em toda a faixa. Os valores normalmente variam entre -60 e 0 dB.                                                 |
| mode             | Int64        | Indica a modalidade (maior ou menor) de uma faixa, com maior representado por 1 e menor por 0.                                                                         |
| speechiness      | Float64      | Detecta a presença de palavras faladas em uma faixa. Valores próximos a 1.0 indicam faixas que provavelmente são compostas inteiramente por palavras faladas.         |
| acousticness     | Float64      | Uma medida de confiança de 0.0 a 1.0 de se a faixa é acústica. 1.0 representa alta confiança de que a faixa é acústica.                                                |
| instrumentalness | Float64      | Prevê se uma faixa não contém vocais. Valores acima de 0.5 pretendem representar faixas instrumentais, mas a confiança é maior à medida que o valor se aproxima de 1.0.|
| liveness         | Float64      | Detecta a presença de uma audiência na gravação. Valores mais altos indicam uma maior probabilidade de que a faixa tenha sido executada ao vivo.                        |
| valence          | Float64      | Uma medida de 0.0 a 1.0 descrevendo a positividade musical transmitida por uma faixa. Faixas com alta valência soam mais positivas (ex. feliz, alegre, eufórica).      |
| tempo            | Float64      | O tempo estimado geral de uma faixa em batidas por minuto (BPM).                                                                                                       |
| genre            | String       | Gênero da faixa.                                                                                                                                                       |



In [2]:
# Leitura do arquivo .csv
df = pl.read_csv('songs_normalize.csv')
df.head(10)

artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
str,str,i64,bool,i64,i64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,str
"""Britney Spears…","""Oops!...I Did …",211160,false,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3,0.0000177,0.355,0.894,95.053,"""pop"""
"""blink-182""","""All The Small …",167066,false,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.0,0.612,0.684,148.726,"""rock, pop"""
"""Faith Hill""","""Breathe""",250546,false,1999,66,0.529,0.496,7,-9.007,1,0.029,0.173,0.0,0.251,0.278,136.859,"""pop, country"""
"""Bon Jovi""","""It's My Life""",224493,false,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.0000135,0.347,0.544,119.992,"""rock, metal"""
"""*NSYNC""","""Bye Bye Bye""",200560,false,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.00104,0.0845,0.879,172.656,"""pop"""
"""Sisqo""","""Thong Song""",253733,true,1999,69,0.706,0.888,2,-6.959,1,0.0654,0.119,0.0000964,0.07,0.714,121.549,"""hip hop, pop, …"
"""Eminem""","""The Real Slim …",284200,true,2000,86,0.949,0.661,5,-4.244,0,0.0572,0.0302,0.0,0.0454,0.76,104.504,"""hip hop"""
"""Robbie William…","""Rock DJ""",258560,false,2000,68,0.708,0.772,7,-4.264,1,0.0322,0.0267,0.0,0.467,0.861,103.035,"""pop, rock"""
"""Destiny's Chil…","""Say My Name""",271333,false,1999,75,0.713,0.678,5,-3.525,0,0.102,0.273,0.0,0.149,0.734,138.009,"""pop, R&B"""


In [3]:
# Verificação de valores nulos no dataframe
ut.nulos(df)


Colunas,Quantidade de nulos
str,i64
"""artist""",0
"""song""",0
"""duration_ms""",0
"""explicit""",0
"""year""",0
"""popularity""",0
"""danceability""",0
"""energy""",0
"""key""",0


In [4]:
# Verificação de valores duplicados no dataframe
ut.duplicadas(df)

Quantidade de linhas únicas: 1941
Quantidade de linhas duplicadas: 59


In [9]:
# Remoção de valores duplicados

df_unique = df.unique()
df_unique.shape

(1941, 18)